#Step 0: Set up

You can use serverless compute to run this notebook or any other classical compute set up

#Introduction

This is the final notebook from the DSA blogpost here: 

We will use DSPy to use everything you just created in the past 3 notebooks. DSPy's declarative framework and dspy.Module allows us to tightly modularize our code and use them interchangeably, not just in this code, but anywhere else. 

For example, you will be create a module to do a function call to a Genie Space. You can take that same module and use it elsewhere with no code change. You will see this in action below 

After this notebook, you will be able to: 
1. Use DSPy to call any LLM regardless of the provider 
2. Create dspy.Signatures to programatically develop instructions for your LLM
3. Create dspy.Modules to put together your entire application 
4. Have a functioning assistant that can use any kind of data about a Patient!

#Install the Dependencies

In [0]:
%pip install --upgrade databricks-sdk mlflow databricks-vectorsearch dspy pillow litellm openai
dbutils.library.restartPython()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/8.0 MB ? eta -:--:--
   ━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/8.0 MB 13.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 7.9/8.0 MB 22.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: litellm
    Found existing installation: litellm 1.72.0
    Uninstalling litellm-1.72.0:
      Successfully uninstalled litellm-1.72.0
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import mlflow
import mlflow.deployments
import base64
import io
from PIL import Image
import dspy
from databricks.vector_search.client import VectorSearchClient

client = mlflow.deployments.get_deploy_client("databricks")
mlflow.dspy.autolog() 

#Configure an LLM provider for DSPy

You can use dspy.LM to specify what LLM you would like to use. If you wish to change what LLM and what provider you want to use, you can use dspy.configure() to change it. No other code needs to be updated

In [0]:
import dspy

claude = dspy.LM('databricks/databricks-claude-3-7-sonnet', cache=False)
claude_anthropic = dspy.LM('anthropic/claude-sonnet-4-20250514', api_key="", cache=False)
# llama = dspy.LM('databricks/databricks-llama-4-maverick', cache=False)
dspy.configure(lm=claude_anthropic)

In [0]:
class DogPictureSignature(dspy.Signature):
    """Output the dog breed of the dog in the image. Describe image"""
    image_1: dspy.Image = dspy.InputField()
    answer: str = dspy.OutputField()

image_url = "https://picsum.photos/id/237/200/300"
classify = dspy.Predict(DogPictureSignature)
classify(image_1=dspy.Image.from_url(image_url))

21:44:23 - LiteLLM:INFO: utils.py:3043 - 
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:LiteLLM:
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:httpx:HTTP Request: GET https://picsum.photos/id/237/200/300 "HTTP/1.1 302 Found"
INFO:httpx:HTTP Request: GET https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
21:44:28 - LiteLLM:INFO: utils.py:1215 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
21:44:28 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: anthropic/claude-sonnet-4-20250514
INFO:LiteLLM:selected model name for cost calculation: anthropic/claude-sonnet-4-20250514
21:44:28 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: anthropic/claude

Prediction(
    answer='This appears to be a Labrador Retriever puppy, specifically a black Lab. The image shows a young black dog with the characteristic features of a Labrador: a broad head, medium-sized ears that hang down, dark expressive eyes, and a sturdy build. The puppy has a glossy black coat and is sitting on what appears to be wooden decking or flooring. Labrador Retrievers are known for their friendly temperament, intelligence, and loyalty, making them one of the most popular family dog breeds. This particular puppy looks to be quite young, likely around 8-12 weeks old based on its size and facial features.'
)

Trace(request_id=tr-aec7eb76243c477091c491f053eb6bb1)

In [0]:
from config import volume_label, volume_name, catalog, schema, model_name, model_endpoint_name, embedding_table_name, embedding_table_name_index, registered_model_name, vector_search_endpoint_name

#Testing our tools

Let's instantiate our tools and test that they still work

##Vector Search Index

In [0]:
vs_client = VectorSearchClient()

# Define the endpoint configuration
vector_search_endpoint_name = "one-env-shared-endpoint-4"
index_name = f"{embedding_table_name}_index"

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


In [0]:
index = vs_client.get_index(endpoint_name=vector_search_endpoint_name, index_name=f"{catalog}.{schema}.{index_name}")

##Vector Search Index + Model Serving Endpoint

In [0]:
input_query = "what is my deductible for united healthcare"

response = client.predict(
            endpoint=model_endpoint_name,
            inputs={"dataframe_split": {
                    "columns": ["text"],
                    "data": [[input_query]]
                    }
            }
          )

query_embedding = response['predictions']['predictions']['embedding']
results = index.similarity_search(num_results=5, columns=["base64_image"], query_vector=query_embedding)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


In [0]:
base64_test_retrieved = results['result']['data_array'][0][0]
print(base64_test_retrieved) #the matching image vector search retrieved

...

*** WARNING: max output size exceeded, skipping output. ***

...


#Check your Genie Space

Change the genie_space_id to the space you created in the 3rd notebook then run the cell. It should return some information about a patient. If it does not work initially, go into your data and find a patient it created as the same patient name may not have been generated

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.dashboards import GenieAPI

def hls_patient_genie(sql_instruction):

  w = WorkspaceClient()
  genie_space_id = "01effef4c7e113f9b8952cf568b49ac7" #replace this with your genie space ID that you created

  # Start a conversation
  conversation = w.genie.start_conversation_and_wait(
      space_id=genie_space_id,
      content=f"{sql_instruction} always limit to one result"
  )

  response = w.genie.get_message_attachment_query_result(
    space_id=genie_space_id,
    conversation_id=conversation.conversation_id,
    message_id=conversation.message_id,
    attachment_id=conversation.attachments[0].attachment_id
  )

  return response.statement_response.result.data_array
hls_result = hls_patient_genie(sql_instruction="Find information about Richard Massey")
print(hls_result)

[['Richard', 'Massey', 'UnitedHealth', 'HMO', None, 'alawrence@example.org', 'LA', '8216', 'Those government continue charge recognize decide seem again.', 'Physical Therapy']]


#DSPy development

Now that we confirmed the resources work. Let's define the dspy.Signatures. 

Signature replace prompt engineering. Your goal for a signature is providing the parameters and defining an input-output relationship. Each signature should be clear on how it is achieving something. 

Each signature can also be used interchangeably with different modules and with each other. 

Below we will define 3 signatures: 
1. Extract_Keywords: To send more relevant words to the vector search index, we will first extract keywords from the text input 
2. Genie_Call: To send the text query to the genie space and understand what insurance type someone has
3. image_analyzer: We will take the results of the vector search index and the original text query, send these to the LLM to interpret and create a response

**Goal**: Identify the deductible of the patient's insurance

**Problem**: Our tables the Genie Space queries does not have a column called deductible. This is located in the PDFs containing insurance information. Based on the input text query, we need to figure out: 
1. Who the Patient is
2. What we are looking for
3. Where to get it 

**Solution**: Let the agent take the text_query, find the patient and the keyword showing what we are looking for. Then, extract these keywords to send to our Genie and Vector Search Tools to find the necessary information. We consolidate this together and let the LLM read through the information to come to a conclusion. 


In [0]:
import time 
class extract_keywords(dspy.Signature):
  "A class to extract keywords from a text query that would work best with a vector search similarity search and find the patient_name"
  text_query: str = dspy.InputField(desc="only words from this text_query should be used") 
  vector_search_keywords: str = dspy.OutputField()
  patient_name: str = dspy.OutputField()

class image_analyzer(dspy.Signature):
  """review the image and genie_patient_response to answer the test_query"""
  image: dspy.Image = dspy.InputField() 
  genie_patient_response: list = dspy.InputField()
  response: str = dspy.OutputField() 

class patient_information_extraction(dspy.Signature):
  """Determine which tool to call to find more information and answer the query"""
  text_query: str = dspy.InputField()
  genie_patient_response: list = dspy.OutputField()
  vector_search_insurance_doc_response: str = dspy.OutputField(desc="base64 string from vector search")

class MultiModalPatientInsuranceAnalyzer(dspy.Module):
  def __init__(self):
    super().__init__()
    # self.insurance_lookup = dspy.ReAct(get_insurance_document, tools=[self.vector_search_for_patient_pdf], max_iters=2)
    self.image_analyzer = dspy.Predict(image_analyzer)
    # self.extract_keywords = dspy.Predict(extract_keywords)
    self.patient_information_extraction = dspy.ReAct(patient_information_extraction, tools=[self.vector_search_for_patient_pdf, self.hls_patient_genie], max_iters=2)
  
  def process_image(self, base64_string):
    image_data = base64.b64decode(base64_string) 
    pil_image = Image.open(io.BytesIO(image_data))
    dspy_image = dspy.Image.from_PIL(pil_image)
    return dspy_image
  
  def vector_search_for_patient_pdf(self, text_query):
    """Pulls matching Insurance Documents based on the text_query"""
    client = mlflow.deployments.get_deploy_client("databricks") 
    response = client.predict(
              endpoint=model_endpoint_name,
              inputs={"dataframe_split": {
                      "columns": ["text"],
                      "data": [[text_query]]
                      }
              }
            )
    text_embedding = response['predictions']['predictions']['embedding']
    index = vs_client.get_index(endpoint_name=vector_search_endpoint_name, index_name=f"{catalog}.{schema}.{index_name}")
    results = index.similarity_search(num_results=3, columns=["base64_image"], query_vector=text_embedding)
    return results['result']['data_array'][0][0]
  
  def hls_patient_genie(self, patient_name):
    """Pull Patient information based on the patient's name"""
    w = WorkspaceClient()
    genie_space_id = "01effef4c7e113f9b8952cf568b49ac7" #replace this with your genie space ID that you created

    # Start a conversation
    conversation = w.genie.start_conversation_and_wait(
        space_id=genie_space_id,
        content=f"Find any details about {patient_name}. Limit your answer to one result."
    )

    response = w.genie.get_message_attachment_query_result(
      space_id=genie_space_id,
      conversation_id=conversation.conversation_id,
      message_id=conversation.message_id,
      attachment_id=conversation.attachments[0].attachment_id
    )

    return response.statement_response.result.data_array


  def forward(self, text_query: str):
    # extract_keywords = self.extract_keywords(text_query=text_query)
    # base64_string = self.vector_search_for_patient_pdf(text_query=extract_keywords.response)
    # dspy_image = self.process_image(base64_string=base64_string)
    # return self.image_analyzer(image=dspy_image, text_query=text_query).response
    results = self.patient_information_extraction(text_query=text_query)
    dspy_image = self.process_image(base64_string=results.vector_search_insurance_doc_response)
    return self.image_analyzer(image=dspy_image, text_query=results.genie_patient_response).response

In [0]:
analyzer = MultiModalPatientInsuranceAnalyzer()
print(analyzer(text_query="what Richard Massey's deductible?"))

22:27:45 - LiteLLM:INFO: utils.py:3043 - 
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:LiteLLM:
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 429 Too Many Requests"
22:27:45 - LiteLLM:INFO: utils.py:3043 - 
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:LiteLLM:
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 429 Too Many Requests"
22:27:47 - LiteLLM:INFO: utils.py:3043 - 
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:LiteLLM:
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 429 Too Many Requests"


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

Downstream, we could use the output and update the Delta Table the Genie Space uses allowing us the use a combination of unstructured and structured data to enrich our data sources!